# PyTorch with Custom Classes and Functions

In [1]:
try:
    import verta
except ImportError:
    !pip install verta

This example features:
- **PyTorch** fully-connected neural network implemented in a separate module (`models/nets.py`)
- **verta**'s Python client logging training results
- **verta**'s Python client logging user-defined modules (`models/nets.py` and `utils/preprocess.py`) for deployment

In [2]:
HOST = "app.verta.ai"

PROJECT_NAME = "MNIST Multiclassification"
EXPERIMENT_NAME = "FC-NN"

In [3]:
# import os
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 

## Imports

In [4]:
from __future__ import print_function

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import itertools
import os
import time

import six

import numpy as np
import pandas as pd

import torch
import torch.optim as optim
import torch.utils.data as data_utils

---

# Log Workflow

## Prepare Data

In [5]:
from utils.data import load_mnist

X, y = load_mnist()

In [6]:
# gather indices to split training data into training and validation sets
shuffled_idxs = np.random.permutation(len(y))
idxs_train = shuffled_idxs[int(len(shuffled_idxs)/10):]  # last 90%
idxs_val = shuffled_idxs[:int(len(shuffled_idxs)/10)]  # first 10%

X_train, y_train = X[idxs_train], y[idxs_train]
X_val, y_val = X[idxs_val], y[idxs_val]

In [7]:
# create Dataset object to support batch training
class TrainingDataset(data_utils.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return (self.features[idx], self.labels[idx])

## Instantiate Client

In [8]:
from verta import Client
from verta.utils import ModelAPI

client = Client(HOST)
proj = client.set_project(PROJECT_NAME)
expt = client.set_experiment(EXPERIMENT_NAME)
run = client.set_experiment_run()

## Define Model

In [9]:
from models.nets import FullyConnected

hidden_size = 512
run.log_hyperparameter("hidden_size", hidden_size)
dropout = 0.2
run.log_hyperparameter("dropout", dropout)

model = FullyConnected(num_features=X.shape[1], hidden_size=hidden_size, dropout=dropout)

## Prepare Hyperparameters

In [10]:
criterion = torch.nn.CrossEntropyLoss()
run.log_hyperparameter("loss_fn", "cross entropy")
optimizer = torch.optim.Adam(model.parameters())
run.log_hyperparameter("optimizer", "adam")

num_epochs = 5
run.log_hyperparameter("num_epochs", num_epochs)
batch_size = 32
run.log_hyperparameter("batch_size", batch_size)

## Run and Log Training

In [11]:
# enable batching of training data
dataset = TrainingDataset(X_train, y_train)
dataloader = data_utils.DataLoader(dataset,
                                   batch_size=batch_size,
                                   shuffle=True)

In [12]:
for i_epoch in range(num_epochs):
    for i_batch, (X_batch, y_batch) in enumerate(dataloader):
        model.zero_grad()  # reset model gradients

        output = model(X_batch)  # conduct forward pass

        loss = criterion(output, y_batch)  # compare model output w/ ground truth
        
        print("\repoch {}/{} | ".format(i_epoch+1, num_epochs), end='')
        print("iteration {}/{} | ".format(i_batch+1, len(dataloader)), end='')
        print("epoch loss avg: {}".format(loss.item()), end='')

        loss.backward()  # backpropogate loss to calculate gradients
        optimizer.step()  # update model weights
    with torch.no_grad():  # no need to calculate gradients when assessing accuracy
        print()
        
        pred_train = model(X_train).numpy().argmax(axis=1)
        train_acc = (pred_train == y_train.numpy()).mean()
        print("Training accuracy: {}".format(train_acc))
        run.log_observation("train_acc", train_acc)
        
        pred_val = model(X_val).numpy().argmax(axis=1)
        val_acc = (pred_val == y_val.numpy()).mean()
        print("Validation accuracy: {}".format(val_acc))
        run.log_observation("val_acc", val_acc)

## Calculate and Log Accuracy on Full Training Set

In [13]:
with torch.no_grad():  # no need to calculate gradients when assessing accuracy
    pred_train = model(X_train).numpy().argmax(axis=1)
    train_acc = (pred_train == y_train.numpy()).mean()
    print("Training accuracy: {}".format(train_acc))
    run.log_metric("train_acc", train_acc)

---

## Log Model for Deployment

`utils/preprocess.py` is used in `models/nets.py`, so it must be uploaded as well.

In [14]:
# create deployment artifacts
custom_modules = ["models/nets.py", "utils/preprocess.py"]
model_api = ModelAPI(X_train.tolist(), model(X_train).tolist())
requirements = ["torch"]

# save and log model
run.log_model(
    model,
    custom_modules=custom_modules,
    model_api=model_api,
)
run.log_requirements(requirements)

---

# Make Live Predictions

## Deploy Model Through Web App

In [15]:
run

## Load Deployed Model

In [16]:
from verta._demo_utils import DeployedModel

deployed_model = DeployedModel(HOST, run.id)

## Query Deployed Model

In [17]:
for x in itertools.cycle(X.tolist()):
    print(deployed_model.predict([x]))
    time.sleep(.5)

---